In [0]:
#importing libraries

import tensorflow as tf
import numpy as np
tf.reset_default_graph()

In [0]:
#Luong_ Attention argumants are . ==> rnn_size (num_units of lstm/gru cell) , 

# encoder_output [batch x max_time x dim] 

# sentence_length ( max _length of sequence in batch)

In [0]:
#let's define encoder first 

batch_size = 4
max_time   = 5
dim        = 12
num_units  = 6
sequence_length = [2,4,5,1]


input_data = np.random.randn(batch_size , max_time , dim).astype(np.float32)

lstm_cell = tf.contrib.rnn.LSTMCell(num_units)

output , last_state = tf.nn.dynamic_rnn(lstm_cell,input_data,sequence_length,dtype=tf.float32)

#input batch x max x dim ==> lstm ==> (batch x max x lstm_dim , batch x lstm_dim )
# return (4, 5, 6) , (4, 6)



# with tf.Session() as sess:
#   sess.run(tf.global_variables_initializer())
#   print(sess.run(last_state[0]).shape)



In [0]:
#now let's Construct the Attention mechanism.


Luong_Attention = tf.contrib.seq2seq.LuongAttention(
                                                  num_units=num_units,
                                                  memory=output,
                                                  memory_sequence_length=sequence_length,
                                                  dtype=None, 
                                                  name='LuongAttention'
                                                 )

#You can test and use it's various methods 
 




# will return [batch_size x alignments_size  ] for next time_stamp

# [ batch_size, alignments_size] (alignments_size is memory's max_time).


In [0]:
#let's print the alignment output 

#for that we need two variable , state and query

query_ = tf.get_variable(name='query_dta_1',
                         shape=[batch_size,num_units],
                         dtype=tf.float32,initializer=tf.random_uniform_initializer(-0.01,0.01))

state_ = tf.get_variable(name='state__dta_l',
                         shape=[batch_size,max_time],
                         dtype=tf.float32,initializer=tf.random_uniform_initializer(-0.01,0.01))

In [133]:
#initial_alignments
initial_alignments = Luong_Attention.initial_alignments(batch_size,dtype=tf.float32)


#initial_state
initial_state   =    Luong_Attention.initial_state(batch_size,dtype=tf.float32)



call = Luong_Attention.__call__(query_,state_)


with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  alignment , next_state = sess.run(call)
  
  print(alignment.shape,next_state.shape)
  
  #batch_size x max_time  alignment _shape
  #batch_size x max_time  next_state_sgape
  
  
  

(4, 5) (4, 5)


In [0]:
#Now we don't need to do those things , Tensorflow have a wrapper called AttentionWrapper which will do all those things

#attention_wrapper arguments ==> cell , attention_mech , rnn_size 

attention_wrapper = tf.contrib.seq2seq.AttentionWrapper(lstm_cell,
                                                        Luong_Attention,
                                                        num_units)

#this will return 

#AttentionWrapperState   batch x num_units
# attention vector       batch x num_units
#current time_stamp      1 , 0 depend of time_staps
#alignments              batch x max_time
# alignment_history
#attention_state         batch x max_time

#     collections.namedtuple("AttentionWrapperState",
#                            ("cell_state", "attention", 
#                                "time", "alignments",
#                             "alignment_history", "attention_state")))

#   """`namedtuple` storing the state of a `AttentionWrapper`.






In [135]:
#let's check and unroll for zero state with timestamp 1

# first we have to define zero_state

zero_s= attention_wrapper.zero_state(batch_size=batch_size,dtype=tf.float32)

wrapper_call = attention_wrapper.__call__(query_,zero_s)
AttentionWrapperState , (cell_state , attention , time , alignments , alignment_history , attention_state) = wrapper_call



with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  AttentionWrapperState , (cell_state , attention , time , alignments , alignment_history , attention_state) =sess.run(wrapper_call)
  

  
  
  print( 'Attention_wrapper_state {} \n '.format(AttentionWrapperState.shape))
  print( 'cell_state {} \n '.format(cell_state))
  print('attention {} \n '.format(attention.shape))
  print('time {} \n ',time)
  print('alignments {} \n '.format(alignments.shape))
  print('alignment_history {} \n ',alignment_history)
  print('attention_state {} \n ',attention_state.shape)
  
  

  
  
  
  
#  Contains:
#     - `cell_state`: The state of the wrapped `RNNCell` at the previous time
#       step.
#     - `attention`: The attention emitted at the previous time step.
#     - `time`: int32 scalar containing the current time step.
#     - `alignments`: A single or tuple of `Tensor`(s) containing the alignments
#        emitted at the previous time step for each attention mechanism.
#     - `alignment_history`: (if enabled) a single or tuple of `TensorArray`(s)
#        containing alignment matrices from all time steps for each attention
#        mechanism. Call `stack()` on each to convert to a `Tensor`.
#     - `attention_state`: A single or tuple of nested objects
#        containing attention mechanism state for each attention mechanism.
#        The objects may contain Tensors or TensorArrays.
#   """



Attention_wrapper_state (4, 6) 
 
cell_state LSTMStateTuple(c=array([[ 3.08043323e-03,  1.55311858e-03,  1.70145626e-03,
         2.11932187e-04,  8.50574463e-04,  2.05059652e-03],
       [-4.74626489e-04,  9.16106452e-04,  1.78456702e-03,
        -1.44209852e-03,  7.57550995e-04, -1.64095720e-03],
       [-3.85492458e-04, -1.17246434e-03,  1.75134250e-04,
        -1.00763515e-04, -4.77696769e-04, -5.45556890e-04],
       [ 2.44189054e-03,  2.35536245e-05,  1.76277978e-03,
         3.67806439e-04,  3.43983644e-04,  1.76626770e-03]], dtype=float32), h=array([[ 1.54447532e-03,  7.77840789e-04,  8.49683536e-04,
         1.06092135e-04,  4.24564350e-04,  1.02856825e-03],
       [-2.36995256e-04,  4.58025868e-04,  8.91984731e-04,
        -7.19621603e-04,  3.78311321e-04, -8.19765264e-04],
       [-1.92408625e-04, -5.86739799e-04,  8.76856939e-05,
        -5.03981573e-05, -2.39042798e-04, -2.72577658e-04],
       [ 1.22316589e-03,  1.17831314e-05,  8.81763175e-04,
         1.84339791e-04,  1

In [136]:
#let's feed this time step for next one 

next_unroll = attention_wrapper.__call__(query_,wrapper_call[1])
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print(sess.run(next_unroll))

  



(array([[ 0.15419605,  0.02215088,  0.09260561,  0.05548464,  0.03287077,
        -0.24773316],
       [ 0.17521137,  0.05044638, -0.15758426, -0.02293995,  0.07392441,
        -0.2799775 ],
       [-0.00876762,  0.12460517,  0.0767331 , -0.04451225, -0.00721595,
         0.02627574],
       [ 0.00121187, -0.13014013,  0.0483407 ,  0.12857303,  0.05783688,
        -0.12511724]], dtype=float32), AttentionWrapperState(cell_state=LSTMStateTuple(c=array([[-0.0517728 , -0.07100058,  0.04009656, -0.01267772,  0.01637715,
         0.01745801],
       [-0.03136916, -0.05405451,  0.01539867, -0.01264702,  0.06239711,
         0.03233024],
       [-0.01143507,  0.01212406, -0.02517536,  0.02526897, -0.02043939,
        -0.01568646],
       [-0.04043157, -0.03508674,  0.07172593, -0.03577641,  0.02680731,
         0.02483258]], dtype=float32), h=array([[-0.02548631, -0.03585832,  0.02078494, -0.00657643,  0.00795086,
         0.00852645],
       [-0.01629876, -0.02728193,  0.00779214, -0.00644129